In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
5. Restart the runtime (Runtime -> Restart Runtime) for any upgraded packages to take effect
"""
# If you're using Google Colab and not running locally, run this cell.

NEMO_DIR_PATH = "NeMo"
BRANCH = 'main'

! git clone https://github.com/NVIDIA/NeMo
%cd NeMo
! python -m pip install git+https://github.com/NVIDIA/NeMo.git@{BRANCH}#egg=nemo_toolkit[asr]
%cd ..

# End-to-end Speaker Diarization with Sortformer

## Sortformer: Bridging the Gap between tokens (ASR) and Timestamps (Diarization)

### Speaker Diarization as a part of ASR system

Speaker diarization is all about figuring out who’s speaking when in an audio recording. In the world of automatic speech recognition (ASR), this becomes even more important for handling conversations with multiple speakers. Multispeaker ASR (also called speaker-attributed or multitalker ASR) uses this process to not just transcribe what’s being said, but also to label each part of the transcript with the right speaker.

As ASR technology continues to advance, speaker diarization is increasingly becoming part of the ASR workflow itself. Some systems now handle speaker labeling and transcription at the same time during decoding. This means you don’t just get accurate text—you're also getting insights into who said what, making it more useful for conversational analysis.

### Challenges in Integrating Speaker Diarization and ASR

However, despite significant advancements, integrating speaker diarization and ASR into a unified, seamless system remains a considerable challenge. A key obstacle lies in the need for extensive high-quality, annotated audio data featuring multiple speakers. Acquiring such data is far more complex than collecting single-speaker audio or image datasets. This challenge is particularly pronounced for low-resource languages and domains like healthcare, where strict privacy regulations further constrain data availability.

On top of that, many real-world use cases need these models to handle really long audio files—sometimes hours of conversation at a time. Training on such lengthy data is even more complicated because it’s hard to find or annotate. This creates a big gap between what’s needed and what’s available, making multispeaker ASR one of the toughest nuts to crack in the field of speech technology.

<img src="images/intro_comparison.png" alt="Intro Comparison" style="width: 800px;"/>


### Sortformer: Simplifying Multispeaker ASR with Arrival Time Sorting

To tackle the complexities of multispeaker automatic speech recognition (ASR), we introduce [*Sortformer*](https://arxiv.org/abs/2409.06656), a new approach that incorporates Sort-Loss and techniques to align timestamps with text tokens. Traditional approaches like permutation-invariant loss (PIL) face challenges when applied in batchable and differentiable computational graphs, especially since token-based objectives struggle to incorporate speaker-specific attributes into PIL-based loss functions.

To address this, we propose an arrival time sorting (ATS) approach. In this method, speaker tokens from ASR outputs and speaker timestamps from diarization outputs are sorted by their arrival times to resolve permutations. This approach allows the multispeaker ASR system to be trained or fine-tuned using token-based cross-entropy loss, eliminating the need for timestamp-based or frame-level objectives with PIL.

<img src="images/ats.png" alt="Arrival Time Sort" style="width: 600px;"/>


The ATS-based multispeaker ASR system is powered by an end-to-end neural diarizer model, Sortformer, which generates speaker-label timestamps in arrival time order (ATO). To train the neural diarizer to produce sorted outputs, we introduce Sort Loss, a method that creates gradients enabling the Transformer model to learn the ATS mechanism.



<img src="images/main_dataflow.png" alt="Main Dataflow" style="width: 500px;"/>

Additionally, as shown in the above figure, our diarization system integrates directly with the ASR encoder. By embedding speaker supervision data as speaker kernels into the ASR encoder states, the system seamlessly combines speaker and transcription information. This unified approach improves performance and simplifies the overall architecture.

As a result, our end-to-end multispeaker ASR system is fully or partially trainable with token objectives, allowing both the ASR and speaker diarization modules to be trained or fine-tuned using these objectives. Additionally, during the multispeaker ASR training phase, no specialized loss calculation functions are needed when using Sortformer, as frameworks for standard single-speaker ASR models can be employed. These compatibilities greatly simplify and accelerate the training and fine-tuning process of multispeaker ASR systems. 

On top of all these benefits, *Sortformer* can be used as a stand-alone end-to-end speaker diarization model. By training a Sortformer diarizer model especially on high-quality simulated data with accurate time-stamps, you can boost the performance of multi-speaker ASR systems, just by integrating the *Sortformer* model as _*Speaker Supervision*_ model in a computation graph.

In this tutorial, we will walk you through the process of training a Sortformer diarizer model with toy dataset. Before starting, we will introduce the concepts of Sort-Loss calculation and the Hybrid loss technique.

## Sort-Loss for *Sortformer* Diarizer

<img src="images/sortformer.png" alt="Sortformer Model with Hybrid Loss" style="width: 500px;"/>

Sort-Loss is designed to compare the predicted outputs with the true labels, typically sorted in arrival-time order or another relevant metric. The key distinction that *Sortformer* introduces compared to previous end-to-end diarization systems such as [EEND-SA](https://arxiv.org/pdf/1909.06247), [EEND-EDA](https://arxiv.org/abs/2106.10654) lies in the organization of class presence $\mathbf{\hat{Y}}$.

The figure below illustrates the difference between Sort-Loss and permutation-invariant loss (PIL) or permutation-free loss.

   - PIL is calculated by finding the permutation of the target that minimizes the loss value between the prediction and the target.

   - Sort-Loss simply compares the arrival-time-sorted version of speaker activity outputs for both the prediction and the target. Note that sometimes the same ground-truth labels lead to different target matrices for Sort-Loss and PIL.

For example, the figure below shows two identical source target matrices (the two matrices at the top), but the resulting target matrices for Sort-Loss and PIL are different.

<img src="images/loss_types.png" alt="PIL model VS SortLoss model" style="width: 1000px;"/>

In mathmatical terms, Sort-Loss can be expressed as follows:

* **Arrival Time Sorting Function with $\Psi$ function**   

    Let $\Psi$ be a function that determines the first segment's arrival time for a speaker bin:
$$
    \Psi\big(\mathbf{y}_{k}\big) = \min \{ t' \mid y_{k, t'} \neq 0, t' \in [1, T] \} = t^{0}_{k},
$$
 where $t^{0}_{k}$ is the frame index of the first active segment for the $k$-th speaker.

Sortformer aims to generate predictions $\mathbf{\hat{y}}_{k}$ for each speaker $k$ such that:
$$
\Psi(\mathbf{\hat{y}}_{1}) \leq \Psi(\mathbf{\hat{y}}_{2}) \leq \cdots \leq \Psi(\mathbf{\hat{y}}_{K}),
$$
ensuring the model produces class presence outputs ($\mathbf{\hat{Y}}$) sorted by arrival time.

* **Sorting Function $\eta$ and Sorted Targets $\mathbf{Y}_{\eta}$**  


Let $\eta$ be the sorting function applied to speaker indices $\{1, \dots, K\}$. The sorted ground-truth matrix $\mathbf{Y}_{\eta}$ is defined as:
$$
\eta\big( \mathbf{Y} \big) = \mathbf{Y}_{\eta} = \left(\mathbf{y}_{\eta(1)}, \dots, \mathbf{y}_{\eta(K)} \right).
$$
Using $\Psi$, the following condition holds for the sorted ground-truth labels $\mathbf{y}_{\eta(k)}$:
$$
\Psi(\mathbf{y}_{\eta(1)}) \leq \Psi(\mathbf{y}_{\eta(2)}) \leq \cdots \leq \Psi(\mathbf{y}_{\eta(K)}).
$$

* **Sort Loss ($\mathcal{L}_{\text{Sort}}$) Definition**  


Sort Loss is computed as:
$$
\mathcal{L}_{\text{Sort}}\left(\mathbf{Y}, \mathbf{P}\right) = \mathcal{L}_{\text{BCE}} \left(\mathbf{Y}_{\eta}, \mathbf{P}\right) = \frac{1}{K} \sum_{k=1}^{K} \mathcal{L}_{\text{BCE}}(\mathbf{y}_{\eta(k)}, \mathbf{q}_k),
$$
where:

- $\mathbf{y}_{\eta(k)}$: True labels sorted by arrival time using the sorting function $\eta$.
- $\mathbf{q}_k$: Predicted outputs for the $k$-th speaker.
- $\mathcal{L}_{\text{BCE}}(\mathbf{y}_{\eta(k)}, \mathbf{q}_k)$: Binary cross-entropy (BCE) loss for the $k$-th speaker.
- $K$: Total number of speakers.


Now that we learn the concept of Sort Loss and Sortformer, we can now calculate Sort Loss based target matrix and PIL-based target matrix to compare the difference in target-value setting atrix and loss calculation.

- raw target matrix $\mathbf{Y}$: `raw_targets`
- prediction matrix $\mathbf{P}$: `preds`
- ATS target matrix $\mathbf{Y}_{\eta}$: `ats_targets`
- PIL target matrix $\mathbf{Y}_{\text{PIL}}$: `pil_targets`

First, assign the values in the above examples to the respective variables to create tensors.

In [ ]:
import torch

# Define the binary grid as a Python list
raw_targets_list = [[
    [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
],]

preds_list = [[
    [1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1],
],]

# Convert the list to a PyTorch tensor
raw_targets = torch.tensor(raw_targets_list).transpose(1,2)
preds = torch.tensor(preds_list).transpose(1,2)

print(raw_targets.shape)
print(preds.shape)

Now, import `get_ats_targets` and `get_pil_targets` functions from the `nemo.collections.asr.parts.utils.asr_multispeaker_utils` module to calculate the ATS and PIL targets. 

In [ ]:
import itertools
import torch
import nemo
from nemo.collections.asr.parts.utils.asr_multispeaker_utils import get_ats_targets, get_pil_targets

max_num_of_spks = 4  # Number of speakers
speaker_inds = list(range(max_num_of_spks))
speaker_permutations = torch.tensor(list(itertools.permutations(speaker_inds)))  # Get all permutations


ats_target = get_ats_targets(labels=raw_targets.clone(), preds=preds, speaker_permutations=speaker_permutations)
pil_target = get_pil_targets(labels=raw_targets.clone(), preds=preds, speaker_permutations=speaker_permutations)

print(f"Predicted tensor:")
print(preds[0].T)

print(f"\nATS target:")
print(ats_target[0].T)

print(f"\nPIL target:")
print(pil_target[0].T)

You can see that ATS target and PIL target are different. Now, you will display the ATS and PIL target matrices to visually compare the difference and also calculate loss values using the BCE loss.

In [ ]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
from nemo.collections.asr.losses.bce_loss import BCELoss 

bce_loss = BCELoss()
# reduction='mean', class_normalization=False)

def plot_diarout(preds, title_text, cmap_str):

    preds_mat = preds.cpu().numpy().transpose()
    grid_color_p = 'gray'
    LW, FS = 0.5, 10

    yticklabels = ["spk0", "spk1", "spk2", "spk3"]
    yticks = np.arange(len(yticklabels))
    fig, axs = plt.subplots(1, 1, figsize=(20, 2))  # 1 row, 2 columns for preds and targets

    axs.imshow(preds_mat, cmap=cmap_str, interpolation='nearest')
    axs.set_title(title_text, fontsize=FS)
    axs.set_xticks(np.arange(-0.5, preds_mat.shape[1], 1), minor=True)
    axs.set_yticks(np.arange(-0.5, preds_mat.shape[0], 1), minor=True)
    axs.set_yticks(yticks)
    axs.set_yticklabels(yticklabels)
    axs.set_xlabel(f"80 ms Frames", fontsize=FS)
    
    # Enable grid
    axs.grid(which='minor', color=grid_color_p, linestyle='-', linewidth=LW)
    axs.tick_params(which="minor", size=0)  # Hide minor ticks
    axs.tick_params(which="major", size=5)  # Show major ticks

    plt.savefig('plot.png', dpi=300) # bbox_inches='tight')
    plt.show()

target_length = torch.tensor([ats_target.shape[1]]) 
print(f"Target length: {target_length}")
plot_diarout(preds[0], title_text='Predictions', cmap_str='viridis')

loss_ats = bce_loss(probs=preds.float(), labels=ats_target.float(), target_lens=target_length)
print(f"[ {loss_ats:.4f} ] is the loss from Arrival Time Sort Target: ")
plot_diarout(ats_target[0], title_text='ATS Target', cmap_str='summer')

loss_pil = bce_loss(probs=preds.float(), labels=pil_target.float(), target_lens=target_length)
print(f"[ {loss_pil:.4f} ] is the loss from Permutation Invariant Loss Target")
plot_diarout(pil_target[0], title_text='PIL Target', cmap_str='inferno')

While Sortformer can be trained solely using Sort Loss, there is a limitation: the arrival time estimation is not always accurate. This issue becomes more pronounced as the number of speakers increases during the training session.

Note that Sortformer models can be trained using Sort Loss only, PIL only, or a hybrid loss by adjusting the weight between these two loss components. The hybrid loss $\mathcal{L}_{\text{hybrid}}$ can be described as follows:


$$
\mathcal{L}_{\text{hybrid}} = \alpha \cdot \mathcal{L}_{\text{Sort}} + \beta \cdot \mathcal{L}_{\text{PIL}},
$$

The weight between ATS and PIL can be adjusted with the variable `model.ats_weight`($\alpha$) and `model.pil_weight`($\beta$) in the YAML file of the Sortformer diarizer model as follows:

```yaml
model: 
  sample_rate: 16000
  pil_weight: 0.5 # Weight for Permutation Invariant Loss (PIL) used in training the Sortformer diarizer model
  ats_weight: 0.5 # Weight for Arrival Time Sort (ATS) loss in training the Sortformer diarizer model
  max_num_of_spks: 4 
  ...
```

## Train a Sortformer Diarizer Model

### Example Data Creation

In this tutorial, we will create a simple toy training dataset using the [NeMo Multispeaker Simulator](https://github.com/NVIDIA/NeMo/blob/main/tutorials/tools/Multispeaker_Simulator.ipynb), with Librispeech as the source dataset for demonstration purposes. If you already have datasets with proper speaker annotations (RTTM files), you can replace the simulated dataset with your own.

If you don’t have access to any speaker diarization datasets, the [NeMo Multispeaker Simulator](https://github.com/NVIDIA/NeMo/blob/main/tutorials/tools/Multispeaker_Simulator.ipynb) can be used to generate a sufficient amount of data samples to meet your requirements.

For more details on the data simulator, refer to the documentation in the [NeMo Multispeaker Simulator](https://github.com/NVIDIA/NeMo/blob/main/tutorials/tools/Multispeaker_Simulator.ipynb). This tutorial will not cover the configurations and detailed process of data simulation.

In [ ]:
# Install dependencies for data simulator
!apt-get install sox libsndfile1 ffmpeg
!pip install wget
!pip install unidecode
!pip install "matplotlib>=3.3.2"

#### Data Simulation Step 1:  Download Required Resources

We need to download the LibriSpeech corpus and corresponding word alignments for generating synthetic multi-speaker audio sessions. In addition, we need to download necessary data cleaning scripts from NeMo git.

In [ ]:
import os
NEMO_DIR_PATH = "NeMo"
BRANCH = 'main'

# download scripts if not already there 
if not os.path.exists('NeMo/scripts'):
  print("Downloading necessary scripts")
  !mkdir -p NeMo/scripts/dataset_processing
  !mkdir -p NeMo/scripts/speaker_tasks
  !wget -P NeMo/scripts/dataset_processing/ https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/scripts/dataset_processing/get_librispeech_data.py
  !wget -P NeMo/scripts/speaker_tasks/ https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/scripts/speaker_tasks/create_alignment_manifest.py
  !wget -P NeMo/scripts/speaker_tasks/ https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/scripts/speaker_tasks/pathfiles_to_diarize_manifest.py

Now that we have downloaded all the necessary scripts for data creation and preparation, we can start the data simulation step by downloading the LibriSpeech corpus.

In [ ]:
!mkdir -p LibriSpeech
!python {NEMO_DIR_PATH}/scripts/dataset_processing/get_librispeech_data.py \
  --data_root LibriSpeech \
  --data_sets dev_clean

We can get the forced word alignments data for the LibriSpeech corpus from [this repository.](https://github.com/CorentinJ/librispeech-alignments). Full forced alignments data can be downloaded at [google drive link for alignments data](https://drive.google.com/file/d/1WYfgr31T-PPwMcxuAq09XZfHQO5Mw8fE/view?usp=sharing). We will download only a subset of forced alignment data containing dev-clean part.

In [ ]:
!wget -nc https://dldata-public.s3.us-east-2.amazonaws.com/LibriSpeech_Alignments.tar.gz
!tar -xzf LibriSpeech_Alignments.tar.gz
!rm -f LibriSpeech_Alignments.tar.gz

#### Data Simulation Step 2:  Produce Manifest File with Forced Alignments

We will merge the LibriSpeech manifest files and LibriSpeech forced alignments into one manifest file for ease of use when generating synthetic data. Create alignment files by running the following script.



In [ ]:
!python {NEMO_DIR_PATH}/scripts/speaker_tasks/create_alignment_manifest.py \
  --input_manifest_filepath LibriSpeech/dev_clean.json \
  --base_alignment_path LibriSpeech_Alignments \
  --output_manifest_filepath ./dev-clean-align.json \
  --ctm_output_directory ./ctm_out \
  --libri_dataset_split dev-clean

#### Data Simulation Step 3:  Set data simulation parameters

Now that we have downloaded all the sources we need for data creation, we need to download data simulator configurations in `.yaml` format. Download the YAML file and download `data_simulator.py` script from NeMo repository.

In [ ]:
from omegaconf import OmegaConf
import os
ROOT = os.getcwd()
conf_dir = os.path.join(ROOT,'conf')
!mkdir -p {conf_dir}
CONFIG_PATH = os.path.join(conf_dir, 'data_simulator.yaml')
if not os.path.exists(CONFIG_PATH):
  !wget -P {conf_dir} https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/tools/speech_data_simulator/conf/data_simulator.yaml

config = OmegaConf.load(CONFIG_PATH)
print(OmegaConf.to_yaml(config))

#### Data Simulation Step 4:   Generate Simulated Audio Session

We will generate a set of example sessions with the following specifications:

- 10 example sessions for train  
- 10 example sessions for validation
- 2-speakers in each session
- 90 seconds of recordings

We need to setup different seed for train and validation sets.

In [ ]:
from nemo.collections.asr.data.data_simulation import MultiSpeakerSimulator

# Generate train set 
ROOT = os.getcwd()
data_dir = os.path.join(ROOT,'simulated_train')
config.data_simulator.random_seed=10
config.data_simulator.manifest_filepath="./dev-clean-align.json"
config.data_simulator.outputs.output_dir=data_dir
config.data_simulator.session_config.num_sessions=10
config.data_simulator.session_config.num_speakers=2
config.data_simulator.session_config.session_length=90
config.data_simulator.background_noise.add_bg=False 

lg = MultiSpeakerSimulator(cfg=config)
lg.generate_sessions()

# Generate validation set 
data_dir = os.path.join(ROOT,'simulated_valid')
config.data_simulator.random_seed=20
config.data_simulator.outputs.output_dir=data_dir

lg = MultiSpeakerSimulator(cfg=config)
lg.generate_sessions()

Now that parameter setting is done, generate the samples by launching `generate_sessions()`.

In [ ]:
lg = MultiSpeakerSimulator(cfg=config)
lg.generate_sessions()

### Data preparation step 5: Listen to and Visualize Session

Listen to the audio and visualize the corresponding speaker timestamps (recorded in a RTTM file for each session).

In [ ]:
import os
import IPython
import matplotlib.pyplot as plt
import numpy as np
import librosa
from nemo.collections.asr.parts.utils.speaker_utils import rttm_to_labels, labels_to_pyannote_object

ROOT = os.getcwd()
data_dir = os.path.join(ROOT,'simulated_train')
audio = os.path.join(data_dir,'multispeaker_session_0.wav')
rttm = os.path.join(data_dir,'multispeaker_session_0.rttm')

sr = 16000
signal, sr = librosa.load(audio,sr=sr) 

fig,ax = plt.subplots(1,1)
fig.set_figwidth(20)
fig.set_figheight(2)
plt.plot(np.arange(len(signal)),signal,'gray')
fig.suptitle('Synthetic Audio Session', fontsize=16)
plt.xlabel('Time (s)', fontsize=18)
plt.yticks([], [])
ax.margins(x=0)
a,_ = plt.xticks()
plt.xticks(a[:-1],a[:-1]/sr);
IPython.display.Audio(audio)

We can visually check the ground-truth file of the first sample by running the following commands. We can see that it has plenty of overlap between two speakers. 

In [ ]:
# display speaker labels for reference
labels = rttm_to_labels(rttm)
reference = labels_to_pyannote_object(labels)
reference

You can check that corresponding RTTM files are generated as ground-truth labels for training and evaluation.

In [ ]:
!cat simulated_train/multispeaker_session_0.rttm | head -10

### Data preparation step 6: Check out the created files

The following files are generated from data simulator:

* _wav files_ (one per audio session) - the output audio sessions
* _rttm files_ (one per audio session) - the speaker timestamps for the corresponding audio session (used for diarization training)
* _list files_ (one per file type per batch of sessions) - a list of generated files of the given type (e.g., wav, rttm), used primarily for manifest creation

Check if the files we need are generated by running the following commands.

In [ ]:
print("\n Training audio files:")
!ls simulated_train/*.wav
print("\n Training audio files:")
!ls simulated_train/*.rttm
print("\n Training RTTM list content:")
!cat simulated_train/synthetic_wav.list
print("\n Training RTTM list content:")
!cat simulated_train/synthetic_rttm.list

print("\n Validation audio files:")
!ls simulated_valid/*.wav
print("\n Validation audio files:")
!ls simulated_valid/*.rttm
print("\n Validation RTTM list content:")
!cat simulated_valid/synthetic_wav.list
print("\n Validation RTTM list content:")
!cat simulated_valid/synthetic_rttm.list

## Prepare Training Data for Sortformer diarizer

Now that we have datasets for both train and validation (dev), we can start preparing and cleaning the data samples for training. Make sure you have the following list of files:

**Training set** 

- Train audio files `.wav`
- A train audio list file `.list`
- Train RTTM files `.rttm`
- A train RTTM list content `.list`

**Validation set**  

- Validation audio files `.wav`
- A validation audio list file `.list`
- Validation RTTM files `.rttm`
- A validation RTTM list file `.list`


Based on these files, we need to create manifest files containing data samples we have. If you don't have a `.list` file, you need to create a `.list` file for the `.wav` files and `.rttm` files.

In [ ]:
# Create a NeMo manifest (.json) file for training dataset
!python {NEMO_DIR_PATH}/scripts/speaker_tasks/pathfiles_to_diarize_manifest.py \
  --add_duration \
  --paths2audio_files='simulated_train/synthetic_wav.list' \
  --paths2rttm_files='simulated_train/synthetic_rttm.list' \
  --manifest_filepath='simulated_train/sortformer_train.json'

# Create a NeMo manifest (.json) file for validation (dev) dataset
!python {NEMO_DIR_PATH}/scripts/speaker_tasks/pathfiles_to_diarize_manifest.py \
  --add_duration \
  --paths2audio_files='simulated_valid/synthetic_wav.list' \
  --paths2rttm_files='simulated_valid/synthetic_rttm.list' \
  --manifest_filepath='simulated_valid/sortformer_valid.json'

If you print the content of the created manifest file, you can see that `.rttm` files in the list and `.wav` files are grouped together in the generated manifest files.

In [ ]:
print("\nTraining Dataset:")
!cat simulated_train/sortformer_train.json | tail -5
print("\nValidation Dataset:")
!cat simulated_valid/sortformer_valid.json | tail -5 

## Train a Sortformer Diarizer Model

### Training an offline Sortformer model

Now that we have prepared all the necessary dataset, we can train an Sortformer diarizer model on the prepared dataset. Download YAML file for training form NeMo repository and load the configuration into `config` variable.

In [ ]:
import nemo
import os
import lightning.pytorch as pl
from omegaconf import OmegaConf
from nemo.collections.asr.models import SortformerEncLabelModel
from nemo.utils.exp_manager import exp_manager

NEMO_ROOT = os.getcwd()
!mkdir -p conf 
!wget -P conf https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/speaker_tasks/diarization/conf/neural_diarizer/sortformer_diarizer_hybrid_loss_4spk-v1.yaml
MODEL_CONFIG = os.path.join(NEMO_ROOT,'conf/sortformer_diarizer_hybrid_loss_4spk-v1.yaml')
config = OmegaConf.load(MODEL_CONFIG)


Setup the `manifest_filepath` for `train_ds` and `validation_ds` by feeding the `json` file paths based on the created training dataset.

In [ ]:
curr_dir = os.getcwd() + "/"
config.model.train_ds.manifest_filepath = f'{curr_dir}simulated_train/sortformer_train.json'
# config.model.test_ds.manifest_filepath = f'{curr_dir}simulated_valid/sortformer_valid.json'
config.model.validation_ds.manifest_filepath = f'{curr_dir}simulated_valid/sortformer_valid.json'
config.trainer.strategy = "ddp_notebook"
config.batch_size = 3

config.trainer.devices=1
config.accelerator="gpu"
print(os.getcwd())

print("config.model.train_ds.manifest_filepath ", config.model.train_ds.manifest_filepath )

Setup a model with the given configuration and start a training session.

In [ ]:

trainer = pl.Trainer(devices=1, accelerator='gpu', max_epochs=50,
                  enable_checkpointing=False, logger=False,
                  log_every_n_steps=5, check_val_every_n_epoch=10)

exp_manager(trainer, config.get("exp_manager", None))
sortformer_model = SortformerEncLabelModel(cfg=config.model, trainer=trainer)
sortformer_model.maybe_init_from_pretrained_checkpoint(config)
trainer.fit(sortformer_model)

### Training a streaming Sortformer model

If you want to train a streaming version of Sortformer, you can download the following YAML file.

In [ ]:

!wget -P conf https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/speaker_tasks/diarization/conf/neural_diarizer/streaming_sortformer_diarizer_4spk-v2.yaml
MODEL_CONFIG = os.path.join(NEMO_ROOT,'conf/streaming_sortformer_diarizer_4spk-v2.yaml')
config = OmegaConf.load(MODEL_CONFIG)

curr_dir = os.getcwd() + "/"
config.model.train_ds.manifest_filepath = f'{curr_dir}simulated_train/sortformer_train.json'
config.model.test_ds.manifest_filepath = f'{curr_dir}simulated_valid/sortformer_valid.json'
config.model.validation_ds.manifest_filepath = f'{curr_dir}simulated_valid/sortformer_valid.json'
config.trainer.strategy = "ddp_notebook"
config.batch_size = 3

config.trainer.devices=1
config.accelerator="gpu"
print(os.getcwd())

print("config.model.train_ds.manifest_filepath ", config.model.train_ds.manifest_filepath )

Initiate a streaming Sortformer diarization training session using the given configurations.

In [ ]:
trainer = pl.Trainer(devices=1, accelerator='gpu', max_epochs=50,
                  enable_checkpointing=False, logger=False,
                  log_every_n_steps=5, check_val_every_n_epoch=10)

exp_manager(trainer, config.get("exp_manager", None))
streaming_sortformer_model = SortformerEncLabelModel(cfg=config.model, trainer=trainer)
streaming_sortformer_model.maybe_init_from_pretrained_checkpoint(config)
trainer.fit(streaming_sortformer_model)